## Analyse all house of commons speeches by current MPs

In [46]:
import spacy
import pandas as pd
import json
from bs4 import BeautifulSoup
from collections import Counter
import requests

from config import TWFY_API_KEY

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-04 14:42:27.082569. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [ ]:
# Get MP -> theyworkforyou_id map
# Probably need to modify after general election to point to previous batch of MPs
mps = pd.read_csv("https://www.theyworkforyou.com/mps/?f=csv")

In [2]:
def get_mp_speeches(mp_id):
    """Get speeches of a particular MP based on TheyWorkForYou id and convert data into long format pandas data frame.
    Each row represents one speech at a particular date and time"""
    all_speeches = pd.DataFrame()
    rows = [1]
    page_no=1
    while len(rows) > 0:
        t = requests.get("https://www.theyworkforyou.com/api/getDebates?key={api_key}&\
                     type=commons&person={person}&results_per_page=1000&num={num}&page={page}&output=js".format(api_key=TWFY_API_KEY,
                                                                                                               person=mp_id,
                                                                                                               num=1000,
                                                                                                               page=page_no))
        rows = t.json()["rows"]
        speeches = []
        # Loop over each row
        for row in rows:
            speeches.append({
                    'speech_id':row["gid"],
                    'speech_url':row["listurl"],
                    'mp_name':row["speaker"]["name"],
                    'mp_constituency':row["speaker"]["constituency"],
                    'mp_party':row["speaker"]["party"],
                    'mp_id':row["person_id"],
                    'date':pd.to_datetime(row["hdate"], format="%Y-%m-%d"),
                    'time':row["htime"],
                    'section_id':row["section_id"],
                    'subsection_id':row["subsection_id"],
                    'debate_title':row["parent"]["body"],
                    'body':BeautifulSoup(row["body"], "html5lib").get_text()
                })
        speeches = pd.DataFrame(speeches)

        # Concatenate onto complete speeches dataframe
        all_speeches = pd.concat([all_speeches, speeches], ignore_index=True)
        # Increment page_counter
        page_no += 1
    
    print("Got speeches for MP {0}".format(mp_id))
    return all_speeches

In [3]:
## Download all MP speeches if this is set to True
if False:
    # Parallelise downloading of MP speeches
    from multiprocessing import Pool

    # Number of threads to use to fetch
    NUM_THREADS = 8
    # Make list of mp ids
    list_of_mp_ids = list(mps["Person ID"])

    # Create pool of threads
    pool = Pool(NUM_THREADS)
    # Use pool.map to download speeches mp by mp
    results = pool.map(get_mp_speeches, list_of_mp_ids)
    pool.close()
    pool.join()

    # Concatenate all mps into one dataframe
    all_mp_speeches = pd.concat(results)
    
    # Write data to a file to save it
    all_mp_speeches.to_hdf("mp_speeches.h5", "speeches")
else:
    ## Read in mp speeches that have been previously downloaded
    all_mp_speeches = pd.read_hdf("mp_speeches.h5", "speeches")

In [4]:
all_mp_speeches["time_"] = pd.to_datetime(all_mp_speeches.time, format="%H:%M:%S", errors="coerce")

In [5]:
all_mp_speeches["time_hour"] = all_mp_speeches.time.str.split(":", expand=True).get(0)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf

In [ ]:
# Load english language model from spacy
nlp = spacy.load("en")

### Run data through NLP

In [7]:
import pyLDAvis
import pyLDAvis.sklearn

In [8]:
# Create new corpus object to store all text data
corpus = textacy.Corpus("en")

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 15:18:56.159192. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [9]:
all_mp_speeches_sample = all_mp_speeches.sample(frac=0.1)

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 15:19:12.606942. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [10]:
del all_mp_speeches

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 15:19:14.054808. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [14]:
all_mp_speeches_metadata = all_mp_speeches_sample.drop("body", 1)
corpus.add_texts(texts=all_mp_speeches_sample["body"], n_threads=8)
del all_mp_speeches_sample # To save memory

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 15:20:59.612693. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [33]:
doc_term_matrix, id2term = textacy.vsm.doc_term_matrix(
     (doc.to_terms_list(named_entities=False, as_strings=True, normalise="lemma", filter_stops=True, filter_punct=True, filter_nums=True)
      for doc in corpus),
     weighting='tfidf', normalize=True, smooth_idf=True, min_df=3, max_df=0.95, max_n_terms=100000)

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 19:04:51.087139. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [44]:
list(corpus[0].to_terms_list(named_entities=True, as_strings=True, normalise="lemma", filter_stops=True, filter_punct=True, filter_nums=True))

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-04 14:11:38.007908. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


['house',
 'speaker',
 '14-minute',
 'back - bench',
 'house',
 'order',
 'remind',
 'house',
 'mr.',
 'speaker',
 'place',
 '14-minute',
 'time',
 'limit',
 'bench',
 'speech',
 'describe',
 'hon',
 'member',
 'late',
 'surge',
 'interest',
 'participate',
 'debate',
 'touch',
 'generous',
 'hope',
 'hon',
 'member',
 'bear',
 'mind',
 'addres',
 'house',
 'mr. speaker',
 '14-minute time',
 'time limit',
 'bench speech',
 'late surge',
 'remind the house',
 'house that mr.',
 'speaker have place',
 'place a 14-minute',
 '14-minute time limit',
 'describe as hon',
 'surge of interest',
 'interest in participate',
 'hope that hon',
 'member will bear',
 'addres the house']

In [35]:
model = textacy.tm.TopicModel('lda', n_topics=30, max_iter=30)
model.fit(doc_term_matrix)

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 19:16:48.781787. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [36]:
doc_topic_matrix = model.transform(doc_term_matrix)

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 20:40:55.289498. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [37]:
top_term_matrix = model.model.components_
doc_lengths = [len(doc) for doc in corpus]
vocab = list(id2term.values())
term_frequency = textacy.vsm.get_term_freqs(doc_term_matrix)

vis_data = pyLDAvis.prepare(
    top_term_matrix,
    doc_topic_matrix,
    doc_lengths,
    vocab,
    term_frequency,
)

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 20:40:55.576741. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [38]:
pyLDAvis.enable_notebook()

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 20:40:56.055377. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [39]:
vis_data

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-05-03 20:40:56.907186. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
26     77.883400        1       1 -1.171276  0.084897
27      6.036961        1       2 -0.188111 -0.503421
20      2.425728        1       3  0.056003 -0.205422
11      1.610313        1       4  0.149303 -0.110580
3       1.133847        1       5 -0.072288 -0.140051
6       1.023648        1       6  0.170438 -0.038216
12      0.799337        1       7  0.144159 -0.002102
1       0.537804        1       8  0.112788  0.025898
18      0.477322        1       9  0.088782  0.032077
22      0.472006        1      10  0.080116  0.034232
9       0.446820        1      11  0.068143  0.036587
29      0.433533        1      12  0.061690  0.037937
0       0.430451        1      13  0.152015  0.010277
8       0.408640        1      14  0.051770  0.039720
21      0.401974        1      15  0.042755  0.040248
5       0.399031        1      16  0.037545  0.041303
17      0.395634        1      17  0.036715  0.041361
10      0.394948        1      18  0.037800  0.040871
24      0.392613        1      19  0.037011  0.041578
2       0.392263        1      20  0.020897  0.047484
23      0.372806        1      21  0.037029  0.041545
4       0.352613        1      22  0.008018  0.046328
19      0.351981        1      23  0.018464  0.043419
13      0.351479        1      24  0.006654  0.044563
16      0.350619        1      25  0.006249  0.044612
25      0.348042        1      26  0.004307  0.044743
15      0.345908        1      27  0.002136  0.044922
14      0.343893        1      28  0.000789  0.045019
7       0.343672        1      29  0.000514  0.045054
28      0.342714        1      30 -0.000413  0.045118, topic_info=      Category          Freq                            Term         Total  \
term                                                                         
432    Default  5.517840e+09                             way  5.517840e+09   
11     Default  1.735113e+10                             hon  1.735113e+10   
157    Default  8.608514e+09                       gentleman  8.608514e+09   
0      Default  3.961925e+09                           order  3.961925e+09   
56     Default  1.004923e+10                           right  1.004923e+10   
94     Default  6.846975e+09                       right hon  6.846975e+09   
58     Default  1.098334e+10                          friend  1.098334e+10   
431    Default  3.436300e+09                            lady  3.436300e+09   
59     Default  1.280590e+10                      government  1.280590e+10   
168    Default  6.661942e+09                           point  6.661942e+09   
185    Default  9.697800e+09                        minister  9.697800e+09   
181    Default  2.079528e+09                             tax  2.079528e+09   
12     Default  1.070554e+10                          member  1.070554e+10   
4      Default  1.608930e+09                         speaker  1.608930e+09   
85     Default  1.027284e+10                          people  1.027284e+10   
180    Default  1.143297e+09                            rise  1.143297e+09   
378    Default  7.365029e+09                            year  7.365029e+09   
78     Default  7.237427e+09                            work  7.237427e+09   
103    Default  7.095951e+09                            bill  7.095951e+09   
3055   Default  6.376183e+08                  point of order  6.376183e+08   
3      Default  3.152431e+09                             mr.  3.152431e+09   
2      Default  8.530581e+09                           house  8.530581e+09   
119    Default  6.749485e+09                              's  6.749485e+09   
413    Default  6.735469e+09                           state  6.735469e+09   
772    Default  1.128085e+09                             yes  1.128085e+09   
492    Default  6.932986e+09                       secretary  6.932986e+09   
983    Default  1.195041e+09                 